In [4]:
# ! pip install -U pip
# ! pip uninstall -y ultralytics  # 若已裝舊版，建議先移除
# ! pip install git+https://github.com/sunsmarterjie/yolov12.git
# ! pip install -U pip setuptools wheel
# ! pip install huggingface_hub transformers safetensors accelerate

##  前處理

In [5]:
import os, glob, subprocess, sys
from pathlib import Path
from ultralytics import YOLO

# ======= 修改成你的實際路徑 =======
TRAINING_IMAGE = r"C:\Users\mike\Desktop\aicup\training_image"
TRAINING_LABEL = r"C:\Users\mike\Desktop\aicup\training_label"
ALIAS_ROOT     = r"C:\Users\mike\Desktop\aicup\alias"   # 會建立 alias\images 與 alias\labels
TRAIN_RANGE    = (1, 35)   # patient0001~0030 → train
VAL_RANGE      = (36, 50)  # patient0031~0050 → val
# =================================

ALIAS_IMG = Path(ALIAS_ROOT) / "images"
ALIAS_LBL = Path(ALIAS_ROOT) / "labels"
Path(ALIAS_ROOT).mkdir(parents=True, exist_ok=True)

def safe_remove(p: Path):
    if p.exists():
        if p.is_symlink():
            p.unlink()
        elif p.is_dir():
            # 先嘗試刪 junction（會當成目錄處理）
            subprocess.run(["cmd", "/c", "rmdir", str(p)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

def make_junction(link: Path, target: Path):
    # 用 Windows 目錄連結（junction）取代 symlink，權限需求低
    cmd = ["cmd", "/c", "mklink", "/J", str(link), str(target)]
    ret = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    if ret.returncode != 0:
        print("[錯誤] 建立目錄連結失敗：", ret.stderr.strip())
        print("請確認目標存在，或改用系統管理員執行。")
        sys.exit(1)

def find_patient_root(root: Path) -> Path:
    for dp, dns, _ in os.walk(root):
        if any(d.startswith("patient") for d in dns):
            return Path(dp)
    return root

# 驗證來源資料夾
for p in [TRAINING_IMAGE, TRAINING_LABEL]:
    if not Path(p).is_dir():
        raise FileNotFoundError(f"找不到資料夾：{p}")

img_root = find_patient_root(Path(TRAINING_IMAGE))
lbl_root = find_patient_root(Path(TRAINING_LABEL))
print("IMG_ROOT =", img_root)
print("LBL_ROOT =", lbl_root)

# 清掉舊連結並建立新的 junction
safe_remove(ALIAS_IMG); safe_remove(ALIAS_LBL)
make_junction(ALIAS_IMG, img_root)
make_junction(ALIAS_LBL, lbl_root)
print(f"[OK] 建立連結：{ALIAS_IMG} → {img_root}")
print(f"[OK] 建立連結：{ALIAS_LBL} → {lbl_root}")

def write_list(p_start, p_end, out_txt):
    total, kept = 0, 0
    recs = []
    for i in range(p_start, p_end + 1):
        patient = f"patient{i:04d}"
        img_dir = ALIAS_IMG / patient
        lbl_dir = ALIAS_LBL / patient
        if not img_dir.is_dir() or not lbl_dir.is_dir():
            continue
        for img_path in glob.glob(str(img_dir / "*.png")):
            total += 1
            base = Path(img_path).stem
            lbl_path = lbl_dir / f"{base}.txt"
            if lbl_path.exists() and lbl_path.stat().st_size > 0:
                recs.append(img_path)   # 注意：清單中保留的是「images」路徑
                kept += 1
    recs.sort()
    Path(out_txt).write_text("\n".join(recs), encoding="utf-8")
    print(f"[OK] {out_txt}：影像{total}，有效配對{kept}")

# 產生 train/val 清單（清單中出現 ...\alias\images\patientXXXX\xxx.png）
write_list(*TRAIN_RANGE, Path(ALIAS_ROOT) / "train.txt")
write_list(*VAL_RANGE,   Path(ALIAS_ROOT) / "val.txt")

print("[完成] 已產生：")
print(" -", Path(ALIAS_ROOT) / "train.txt")
print(" -", Path(ALIAS_ROOT) / "val.txt")



Exception in thread Thread-16 (_readerthread):
Traceback (most recent call last):
  File "C:\Users\mike\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "c:\Users\mike\Desktop\aicup\.venv\Lib\site-packages\ipykernel\ipkernel.py", line 788, in run_closure
    _threading_Thread_run(self)
  File "C:\Users\mike\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\mike\AppData\Local\Programs\Python\Python312\Lib\subprocess.py", line 1599, in _readerthread
    buffer.append(fh.read())
                  ^^^^^^^^^
  File "<frozen codecs>", line 322, in decode
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa4 in position 0: invalid start byte
Exception in thread Thread-18 (_readerthread):
Traceback (most recent call last):
  File "C:\Users\mike\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1075, in _bootstrap_inner
    sel

IMG_ROOT = C:\Users\mike\Desktop\aicup\training_image
LBL_ROOT = C:\Users\mike\Desktop\aicup\training_label
[OK] 建立連結：C:\Users\mike\Desktop\aicup\alias\images → C:\Users\mike\Desktop\aicup\training_image
[OK] 建立連結：C:\Users\mike\Desktop\aicup\alias\labels → C:\Users\mike\Desktop\aicup\training_label
[OK] C:\Users\mike\Desktop\aicup\alias\train.txt：影像11470，有效配對1896
[OK] C:\Users\mike\Desktop\aicup\alias\val.txt：影像5393，有效配對891
[完成] 已產生：
 - C:\Users\mike\Desktop\aicup\alias\train.txt
 - C:\Users\mike\Desktop\aicup\alias\val.txt


# 測試測試

In [6]:
# === 區塊 0：環境與路徑設定 ===
# 目的：
# 1) 設定專案、資料、輸出目錄
# 2) 鎖定隨機性（可重現）
# 3) 檢查必要套件（ultralytics）是否可用

import os
import random
import json
import math
from pathlib import Path
from collections import defaultdict
import numpy as np
import pandas as pd

# --- 可自訂參數（依你的環境調整） ---
PROJECT_ROOT = Path.cwd()                 # 預設為當前 Notebook 所在目錄
IMAGES_DIR   = r"C:\Users\mike\Desktop\aicup\alias\images"       # 影像：*.png（512x512）
LABELS_DIR   = r"C:\Users\mike\Desktop\aicup\alias\labels"       # 標註：YOLO .txt（同名）
FOLDS_DIR    = PROJECT_ROOT / "folds"     # 這次新增的 5-fold 清單輸出目錄
RUNS_DIR     = PROJECT_ROOT / "runs"      # 訓練輸出（權重、log）
SUBMIT_DIR   = PROJECT_ROOT / "submissions"

# ultralytics（YOLO）若沒裝，請先安裝：pip install ultralytics
try:
    from ultralytics import YOLO
    _yolo_ok = True
except Exception as e:
    print("⚠️ 未偵測到 ultralytics，訓練/推論區塊會需要它。", e)
    _yolo_ok = False

# --- 亂數種子：鎖定所有主要來源 ---
SEED = 42
random.seed(SEED); np.random.seed(SEED)

import torch
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True  # 較穩定但可能略慢
torch.backends.cudnn.benchmark = False

# --- 目錄建立 ---
for d in [FOLDS_DIR, RUNS_DIR, SUBMIT_DIR]:
    d.mkdir(parents=True, exist_ok=True)

print("專案根目錄：", PROJECT_ROOT)
print("影像：", IMAGES_DIR)
print("標註：", LABELS_DIR)


專案根目錄： c:\Users\mike\Desktop\aicup
影像： C:\Users\mike\Desktop\aicup\alias\images
標註： C:\Users\mike\Desktop\aicup\alias\labels


In [7]:
# === 區塊 1：建立影像中繼資訊表 ===
# 假設影像命名規則可解析出 patient_id 與 slice_idx
# 常見命名範例：
#   images/patient0001/000123.png  -> patient_id = patient0001, slice_idx = 123
# 若你的命名不同，改寫 parse 函式即可。

from typing import Tuple

def parse_patient_and_slice(p: Path) -> Tuple[str, int]:
    """
    傳回 (patient_id, slice_idx)
    - patient_id: 父資料夾名或檔名中可辨識的段
    - slice_idx: 從檔名取整數（不含副檔名）
    """
    # 方案A：以父資料夾為病人ID、檔名（去副檔名）為切片index
    patient_id = p.parent.name
    try:
        slice_idx = int(p.stem)
    except:
        # 若檔名不是純數字，可自訂解析規則
        # 例如 "CT_00123.png" -> 提取 "00123"
        digits = ''.join(ch for ch in p.stem if ch.isdigit())
        slice_idx = int(digits) if digits else -1
    return patient_id, slice_idx

def has_label_for(image_path: Path) -> bool:
    """
    YOLO 規則：標註檔與影像同名、位於 labels/ 對應層級。
    例：images/patient0001/000123.png -> labels/patient0001/000123.txt
    檢查該 .txt 是否存在且至少有一行（存在bbox）
    """
    rel = image_path.relative_to(IMAGES_DIR).with_suffix(".txt")
    label_path = LABELS_DIR / rel
    if not label_path.exists():
        return False
    try:
        # 至少一行標註才算「正樣本」
        return label_path.read_text(encoding="utf-8").strip() != ""
    except:
        return False

# 確保 IMAGES_DIR 和 LABELS_DIR 是 Path 物件
IMAGES_DIR = Path(IMAGES_DIR)
LABELS_DIR = Path(LABELS_DIR)

# 掃描所有影像
all_images = sorted(IMAGES_DIR.rglob("*.png"))
rows = []
for img in all_images:
    pid, sidx = parse_patient_and_slice(img)
    rows.append({
        "img_path": str(img),
        "patient_id": pid,
        "slice_idx": sidx,
        "has_label": has_label_for(img)
    })
df = pd.DataFrame(rows).sort_values(["patient_id", "slice_idx"]).reset_index(drop=True)

# 基本統計
per_patient = df.groupby("patient_id").agg(
    n_images=("img_path", "count"),
    n_pos=("has_label", "sum")
).reset_index()
per_patient["pos_ratio"] = per_patient["n_pos"] / per_patient["n_images"]

print("總影像數：", len(df))
print("總正樣本數：", df["has_label"].sum())
print("病人數：", df["patient_id"].nunique())
display(per_patient.head(10))


總影像數： 16863
總正樣本數： 2787
病人數： 50


,patient_id,n_images,n_pos,pos_ratio
0,patient0001,341,50,0.146628
1,patient0002,391,65,0.166240
2,patient0003,324,70,0.216049
3,patient0004,365,71,0.194521
4,patient0005,285,54,0.189474
5,patient0006,277,47,0.169675
6,patient0007,389,65,0.167095
7,patient0008,287,38,0.132404
8,patient0009,298,42,0.140940
9,patient0010,280,62,0.221429


In [8]:
!pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [9]:
# === 區塊 2：建立病人級 5-fold ===
# 用 KFold 對「病人ID」做分割，每個 fold 互斥的病人集合。

from sklearn.model_selection import KFold

patient_ids = sorted(df["patient_id"].unique().tolist())
assert len(patient_ids) == 50, "⚠️ 病人數應為 50，請檢查資料或命名解析。"

kf = KFold(n_splits=5, shuffle=True, random_state=SEED)
fold_assign = {}  # fold_id -> {"train": [...], "val": [...]}

for fold_id, (tr_idx, va_idx) in enumerate(kf.split(patient_ids), start=1):
    tr_patients = [patient_ids[i] for i in tr_idx]
    va_patients = [patient_ids[i] for i in va_idx]
    fold_assign[fold_id] = {"train": tr_patients, "val": va_patients}

# 儲存 JSON 以利重現
with open(FOLDS_DIR / "patient_folds.json", "w", encoding="utf-8") as f:
    json.dump(fold_assign, f, ensure_ascii=False, indent=2)

# 依病人ID，輸出各 fold 的原始（未擴充）train/val 影像清單
for fold_id, dct in fold_assign.items():
    tr = df[df["patient_id"].isin(dct["train"])]
    va = df[df["patient_id"].isin(dct["val"])]
    (FOLDS_DIR / f"fold{fold_id}_train.txt").write_text("\n".join(tr["img_path"]), encoding="utf-8")
    (FOLDS_DIR / f"fold{fold_id}_val.txt").write_text("\n".join(va["img_path"]), encoding="utf-8")

print("✅ 已建立 5-fold 病人級分配與基本清單")


✅ 已建立 5-fold 病人級分配與基本清單


In [10]:
# === 區塊 3：為訓練集加入鄰近片難負樣本 ===
# 想法：
# - 對每個正樣本（有標註的切片），收集同病人的 ±1、±2、±3 切片
# - 這些鄰近片若無標註 -> 設為「難負樣本」
# - 控制整體負:正比例 ≈ 2~3 : 1，避免負樣本淹沒正樣本訊號
# - 驗證/測試集不做任何擴充（避免高估）

NEIGHBOR_OFFSETS = [-3, -2, -1, 1, 2, 3]
TARGET_NEG_POS_RATIO = 2.5  # 你可調 2~3

def build_neighbors_for_fold(fold_id: int):
    # 取該 fold 的 train/val 病人
    tr_p = set(fold_assign[fold_id]["train"])
    va_p = set(fold_assign[fold_id]["val"])

    # 拆出 train / val DataFrame
    dtr = df[df["patient_id"].isin(tr_p)].copy()
    dva = df[df["patient_id"].isin(va_p)].copy()

    # 建 index：每個病人 -> 切片索引 -> 該 row
    by_patient = {pid: pdf.sort_values("slice_idx").reset_index(drop=True)
                  for pid, pdf in dtr.groupby("patient_id")}

    # 找出 train 中的正樣本（基底集合）
    pos_rows = dtr[dtr["has_label"]].copy()
    n_pos = len(pos_rows)

    # 收集鄰近片候選（僅限無標註的）
    hard_negs = []
    for _, r in pos_rows.iterrows():
        pid, sidx = r["patient_id"], r["slice_idx"]
        pdf = by_patient[pid]
        # 在當病人的切片表找相鄰 index
        for off in NEIGHBOR_OFFSETS:
            target_idx = sidx + off
            cand = pdf[pdf["slice_idx"] == target_idx]
            if len(cand) == 1:
                cr = cand.iloc[0]
                if not cr["has_label"]:   # 只有沒有標註的才納入難負樣本
                    hard_negs.append(cr["img_path"])

    # 去重
    hard_negs = sorted(set(hard_negs))

    # 其餘的負樣本（容易負樣本）：train 全部無標註 - 已選 hard_negs
    all_neg = set(dtr[~dtr["has_label"]]["img_path"].tolist())
    easy_negs = sorted(all_neg - set(hard_negs))

    # 決定最終要多少負樣本（按目標比例）
    n_neg_target = int(TARGET_NEG_POS_RATIO * n_pos)

    # 先放入所有 hard_negs，若不足再從 easy_negs 補
    selected_negs = hard_negs.copy()
    if len(selected_negs) < n_neg_target:
        need = n_neg_target - len(selected_negs)
        random.shuffle(easy_negs)
        selected_negs += easy_negs[:need]
    else:
        # 若 hard_negs 已超量，隨機下采樣到目標數
        random.shuffle(selected_negs)
        selected_negs = selected_negs[:n_neg_target]

    # 正樣本影像（所有有標註切片）
    selected_pos = pos_rows["img_path"].tolist()

    # 合併最終 train 清單（打散順序較佳）
    final_train = selected_pos + selected_negs
    random.shuffle(final_train)

    # 輸出檔案
    out_path = FOLDS_DIR / f"fold{fold_id}_train_neighbors.txt"
    out_path.write_text("\n".join(final_train), encoding="utf-8")

    # 統計報表
    stats = {
        "fold": fold_id,
        "n_train_total": len(dtr),
        "n_pos_all": int(dtr["has_label"].sum()),
        "n_pos_used": len(selected_pos),
        "n_hard_neg": len(hard_negs),
        "n_easy_neg_used": len(selected_negs) - len(hard_negs),
        "n_final_train": len(final_train),
        "neg_pos_ratio_final": round(len(selected_negs) / max(1, len(selected_pos)), 3)
    }
    return stats

all_stats = []
for k in range(1, 6):
    stats_k = build_neighbors_for_fold(k)
    all_stats.append(stats_k)

pd.DataFrame(all_stats)


,fold,n_train_total,n_pos_all,n_pos_used,n_hard_neg,n_easy_neg_used,n_final_train,neg_pos_ratio_final
0,1,13820,2251,2251,240,5387,7878,2.5
1,2,13266,2152,2152,240,5140,7532,2.5
2,3,13551,2250,2250,240,5385,7875,2.5
3,4,13344,2232,2232,240,5340,7812,2.5
4,5,13471,2263,2263,240,5417,7920,2.5


In [11]:
# === 區塊 4：訓練全域參數（易於切換） ===
# 這些參數會在訓練迴圈被引用。視你的 GPU/時間自行調整。

IMG_SIZE      = 640   # 小物件建議提升解析度；若爆顯存，可改 896
EPOCHS        = 40    # 100~150 皆可，平衡時間與效果
BATCH_SIZE    = 8      # 視顯存調整
ACCUMULATE    = 2      # 累積梯度 -> 提高有效 batch（BATCH_SIZE * ACCUMULATE）
BASE_LR       = 1e-3   # 初始學習率（可依模型/資料微調）
WEIGHT_DECAY  = 1e-3
USE_AMP       = True   # 混合精度
USE_P2_HEAD   = True  # 若你有支援P2特徵的模型變體，可改 True

# 損失/訓練細節（ultralytics 有些以 args 或 model.yaml 控）
# 下列僅示意：實際要看你選用的模型與版本支持的參數
TRAIN_ARGS = dict(
    imgsz=IMG_SIZE,
    epochs=EPOCHS,
    batch=BATCH_SIZE,
    lr0=BASE_LR,
    weight_decay=WEIGHT_DECAY,
    optimizer='AdamW',
    patience=10,      # 早停耐心值
    cos_lr=True,      # Cosine LR
    amp=USE_AMP,
    device=0 
)

print("訓練參數：", TRAIN_ARGS)


訓練參數： {'imgsz': 640, 'epochs': 40, 'batch': 8, 'lr0': 0.001, 'weight_decay': 0.001, 'optimizer': 'AdamW', 'patience': 10, 'cos_lr': True, 'amp': True, 'device': 0}


In [12]:
# === 區塊 5：資料 YAML 覆寫策略 ===
# 你的 aortic_val.yaml 依然存在，但 train/val 清單由 Notebook 動態替換。
# 這裡組裝一個臨時 YAML 字串，供 YOLO 調用。

import yaml

def make_data_yaml(train_list_path: Path, val_list_path: Path) -> Path:
    data_dict = {
        "path": str(r"C:\Users\mike\Desktop\aicup\alias"),    # 資料根目錄（可讓YOLO拼相對路徑）
        "train": str(train_list_path),
        "val": str(val_list_path),
        "nc": 1,
        "names": {0: "aortic_valve"}
    }
    tmp_yaml = PROJECT_ROOT / f"data_tmp_{train_list_path.stem}.yaml"
    with open(tmp_yaml, "w", encoding="utf-8") as f:
        yaml.safe_dump(data_dict, f, allow_unicode=True, sort_keys=False)
    return tmp_yaml

print("✅ 資料 YAML 會在每個 fold 動態產生")


✅ 資料 YAML 會在每個 fold 動態產生


In [13]:
# === 區塊 6：5-fold 訓練 ===
# 你可以選擇單一 fold 測試或全部跑完。
# 模型選擇：
#   - 若你有自訂的 model YAML（支援P2），替換 'yolov8n.pt' 為你的權重或架構。
#   - 初期可用輕量模型驗證流程是否正確，再換大模型衝分。

if not _yolo_ok:
    raise SystemExit("請先安裝 ultralytics 才能執行訓練。")

fold_results = []

for fold_id in range(1, 6):
    train_list = FOLDS_DIR / f"fold{fold_id}_train_neighbors.txt"
    val_list   = FOLDS_DIR / f"fold{fold_id}_val.txt"
    data_yaml  = make_data_yaml(train_list, val_list)

    save_dir = RUNS_DIR / f"fold{fold_id}"
    save_dir.mkdir(parents=True, exist_ok=True)

    print(f"\n=== 開始訓練：Fold {fold_id} ===")
    print("Train 清單：", train_list)
    print("Val 清單：", val_list)

    MODEL_PATH = "yolov12s.pt"  # 依你資源改 n/s/m/l/x；先中杯測效果
    model = YOLO(MODEL_PATH)

    # 開始訓練
    res = model.train(
        data=str(data_yaml),
        project=str(save_dir),
        name="exp",
        **TRAIN_ARGS
    )

    # 取最佳權重路徑（ultralytics 預設保存在 runs/...）
    best_weight = save_dir / "exp" / "weights" / "best.pt"
    if not best_weight.exists():
        # 有些版本會在 expN 資料夾，保守搜尋
        best_candidates = list(save_dir.rglob("best.pt"))
        best_weight = best_candidates[0] if best_candidates else None

    fold_results.append({
        "fold": fold_id,
        "best_weight": str(best_weight) if best_weight else ""
    })

pd.DataFrame(fold_results)



=== 開始訓練：Fold 1 ===
Train 清單： c:\Users\mike\Desktop\aicup\folds\fold1_train_neighbors.txt
Val 清單： c:\Users\mike\Desktop\aicup\folds\fold1_val.txt
New https://pypi.org/project/ultralytics/8.3.215 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.63  Python-3.12.10 torch-2.9.0+cu128 CUDA:0 (NVIDIA GeForce RTX 5070 Ti, 16303MiB)
engine\trainer: task=detect, mode=train, model=yolov12s.pt, data=c:\Users\mike\Desktop\aicup\data_tmp_fold1_train_neighbors.yaml, epochs=40, time=None, patience=10, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=c:\Users\mike\Desktop\aicup\runs\fold1, name=exp, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None

train: Scanning C:\Users\mike\Desktop\aicup\alias\labels\patient0001... 2251 images, 5627 backgrounds, 0 corrupt: 100%|██████████| 7878/7878 [00:03<00:00, 1984.44it/s]


train: New cache created: C:\Users\mike\Desktop\aicup\alias\labels\patient0001.cache


val: Scanning C:\Users\mike\Desktop\aicup\alias\labels\patient0014.cache... 536 images, 2507 backgrounds, 0 corrupt: 100%|██████████| 3043/3043 [00:00<?, ?it/s]


Plotting labels to c:\Users\mike\Desktop\aicup\runs\fold1\exp\labels.jpg... 
optimizer: AdamW(lr=0.001, momentum=0.937) with parameter groups 121 weight(decay=0.0), 128 weight(decay=0.001), 127 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to c:\Users\mike\Desktop\aicup\runs\fold1\exp
Starting training for 40 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all       3392        524      0.863      0.886      0.933      0.705



40 epochs completed in 2.009 hours.
Optimizer stripped from c:\Users\mike\Desktop\aicup\runs\fold5\exp\weights\last.pt, 18.6MB
Optimizer stripped from c:\Users\mike\Desktop\aicup\runs\fold5\exp\weights\best.pt, 18.6MB

Validating c:\Users\mike\Desktop\aicup\runs\fold5\exp\weights\best.pt...
Ultralytics 8.3.63  Python-3.12.10 torch-2.9.0+cu128 CUDA:0 (NVIDIA GeForce RTX 5070 Ti, 16303MiB)
YOLOv12s summary (fused): 376 layers, 9,074,595 parameters, 0 gradients, 19.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 212/212 [00:11<00:00, 18.84it/s]


                   all       3392        524      0.863      0.887      0.933      0.705
Speed: 0.2ms preprocess, 2.0ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to c:\Users\mike\Desktop\aicup\runs\fold5\exp


,fold,best_weight
0,1,c:\Users\mike\Desktop\aicup\runs\fold1\exp\wei...
1,2,c:\Users\mike\Desktop\aicup\runs\fold2\exp\wei...
2,3,c:\Users\mike\Desktop\aicup\runs\fold3\exp\wei...
3,4,c:\Users\mike\Desktop\aicup\runs\fold4\exp\wei...
4,5,c:\Users\mike\Desktop\aicup\runs\fold5\exp\wei...


In [14]:
# === 區塊 7（改版）：對兩個測試資料夾推論並合併結果 ===
# 需求：
# - 你的測試資料夾在 Windows：
#     C:\Users\307\Desktop\aicup\test\images1
#     C:\Users\307\Desktop\aicup\test\images2
# - 這裡會：
#   1) 掃描兩個資料夾所有 .png 影像
#   2) 用已訓練好的模型權重做推論（可選擇單一 fold 或多 fold）
#   3) 把兩個資料夾的推論結果「合併成同一個 preds 字典」

from pathlib import Path
import os

# ---- 你要推論的兩個測試資料夾（Windows 路徑，用 r"..." 原始字串避免跳脫字元問題）----
TEST_DIRS = [
    Path(r"C:\Users\mike\Desktop\aicup\test\images1"),
    Path(r"C:\Users\mike\Desktop\aicup\test\images2")
]

# ---- 選擇推論用的權重 ----
# 方案 A：只用某一個 fold 的 best.pt（最簡單，跑最少）
USE_ENSEMBLE = True     # 若要改用多 fold 融合，設 True（下方有簡易投票版）

# 如果只用單一 fold，指定 fold_id；若用 Ensemble，下方會自動抓全部 folds 的 best.pt
SINGLE_FOLD_ID = 1

# ---- 輔助：列出資料夾內所有 .png 影像（遞迴掃描）----
def list_pngs(folder: Path):
    if not folder.exists():
        raise FileNotFoundError(f"找不到測試資料夾：{folder}")
    return sorted([p for p in folder.rglob("*.png")])

# ---- 輔助：把圖片路徑清單轉成 stem 清單（不含副檔名），之後提交檔會用到 ----
def stems_from_paths(paths):
    return [p.stem for p in paths]

# ---- 推論函式（沿用你前面區塊的 run_inference，但這裡接受「路徑清單」而非 .txt）----
def run_inference_on_paths(weight_path: Path, img_paths: list, conf: float = 0.1):
    """
    以 YOLO 權重對 img_paths（路徑清單）推論
    回傳 dict: img_stem -> [ (cls, score, x1,y1,x2,y2), ... ]
    """
    from ultralytics import YOLO
    mdl = YOLO(str(weight_path))
    results_dict = {}

    for img in img_paths:
        res = mdl.predict(source=str(img), imgsz=IMG_SIZE, conf=conf, verbose=False)[0]

        dets = []
        if res and res.boxes is not None and len(res.boxes) > 0:
            for b in res.boxes:
                xyxy = b.xyxy.cpu().numpy().astype(float)[0]
                sc   = float(b.conf.cpu().numpy()[0])
                cl   = int(b.cls.cpu().numpy()[0])
                x1, y1, x2, y2 = [int(round(v)) for v in xyxy]
                dets.append((cl, sc, x1, y1, x2, y2))
        results_dict[Path(img).stem] = dets
    return results_dict

# ---- 如果開啟 Ensemble：簡單的「分數平均 + NMS」融合（輕量示範版）----
# 說明：為了不引入外部 WBF 套件，我們用簡化版融合：
# - 對同一張圖，把多個模型的框全部收集
# - 針對重疊(IOU >= 0.55)的框做分組，分數取平均、座標取平均
# - 這版不如 WBF 嚴謹，但易於整合、0 依賴；要更強可改用 ensemble-boxes/WBF
import math

def iou_xyxy(a, b):
    ax1, ay1, ax2, ay2 = a
    bx1, by1, bx2, by2 = b
    inter_x1 = max(ax1, bx1); inter_y1 = max(ay1, by1)
    inter_x2 = min(ax2, bx2); inter_y2 = min(ay2, by2)
    iw = max(0, inter_x2 - inter_x1); ih = max(0, inter_y2 - inter_y1)
    inter = iw * ih
    area_a = max(0, (ax2 - ax1)) * max(0, (ay2 - ay1))
    area_b = max(0, (bx2 - bx1)) * max(0, (by2 - by1))
    union = area_a + area_b - inter
    return inter / union if union > 0 else 0.0

def simple_ensemble_merge(list_of_preds_dicts, iou_thr=0.55):
    """
    list_of_preds_dicts: [preds_dict_model1, preds_dict_model2, ...]
      其中每個 preds_dict: img_stem -> [(cls, score, x1,y1,x2,y2), ...]
    回傳同格式的 preds_merged
    """
    merged = {}
    # 先列出所有影像的 key（stem 的聯集）
    all_stems = set()
    for d in list_of_preds_dicts:
        all_stems.update(d.keys())

    for stem in all_stems:
        # 收集此圖全部模型的框
        all_boxes = []
        for d in list_of_preds_dicts:
            all_boxes.extend(d.get(stem, []))  # [(cls,score,x1,y1,x2,y2),...]

        # 以簡易群聚方式融合
        used = [False] * len(all_boxes)
        fused = []
        for i, bi in enumerate(all_boxes):
            if used[i]: 
                continue
            cls_i, sc_i, x1i, y1i, x2i, y2i = bi
            group = [(cls_i, sc_i, x1i, y1i, x2i, y2i)]
            used[i] = True
            for j in range(i+1, len(all_boxes)):
                if used[j]:
                    continue
                cls_j, sc_j, x1j, y1j, x2j, y2j = all_boxes[j]
                # 只融合同類別且 IOU 達閾值的框
                if cls_i == cls_j and iou_xyxy((x1i,y1i,x2i,y2i), (x1j,y1j,x2j,y2j)) >= iou_thr:
                    group.append((cls_j, sc_j, x1j, y1j, x2j, y2j))
                    used[j] = True
            # 對 group 做平均（座標取平均，分數取平均）
            if len(group) == 1:
                fused.append(group[0])
            else:
                cls_g = group[0][0]
                scores = [g[1] for g in group]
                xs1 = [g[2] for g in group]; ys1 = [g[3] for g in group]
                xs2 = [g[4] for g in group]; ys2 = [g[5] for g in group]
                sc_m = sum(scores)/len(scores)
                x1_m = int(round(sum(xs1)/len(xs1))); y1_m = int(round(sum(ys1)/len(ys1)))
                x2_m = int(round(sum(xs2)/len(xs2))); y2_m = int(round(sum(ys2)/len(ys2)))
                fused.append((cls_g, sc_m, x1_m, y1_m, x2_m, y2_m))
        merged[stem] = fused
    return merged

# ---- 1) 掃描兩個資料夾的圖片 ----
all_test_paths = []
for td in TEST_DIRS:
    imgs = list_pngs(td)
    print(f"測試資料夾：{td}，找到 {len(imgs)} 張 PNG")
    all_test_paths.extend(imgs)

print(f"兩資料夾合計：{len(all_test_paths)} 張")

# ---- 2) 跑推論（單一 fold 或 多 fold）----
if not USE_ENSEMBLE:
    # 單 fold（簡單直接）
    weight_path = Path(fold_results[SINGLE_FOLD_ID-1]['best_weight'])
    assert weight_path.exists(), f"找不到指定 fold 的權重：{weight_path}"
    preds_merged = run_inference_on_paths(weight_path, all_test_paths, conf=0.05)
else:
    # Ensemble（把每個 fold 的 best.pt 都跑一遍，最後融合）
    preds_list = []
    for fr in fold_results:
        w = Path(fr["best_weight"])
        if not w.exists():
            print("⚠️ 跳過不存在的權重：", w)
            continue
        print("Ensemble 推論 using:", w)
        preds_list.append(run_inference_on_paths(w, all_test_paths, conf=0.05))
    assert len(preds_list) > 0, "沒有可用的權重可做 Ensemble。"
    preds_merged = simple_ensemble_merge(preds_list, iou_thr=0.55)

# preds_merged 的格式：{img_stem: [(cls, score, x1,y1,x2,y2), ...], ...}
print("示例：", list(preds_merged.items())[:1])


測試資料夾：C:\Users\mike\Desktop\aicup\test\images1，找到 8310 張 PNG
測試資料夾：C:\Users\mike\Desktop\aicup\test\images2，找到 8310 張 PNG
兩資料夾合計：16620 張
Ensemble 推論 using: c:\Users\mike\Desktop\aicup\runs\fold1\exp\weights\best.pt
Ensemble 推論 using: c:\Users\mike\Desktop\aicup\runs\fold2\exp\weights\best.pt
Ensemble 推論 using: c:\Users\mike\Desktop\aicup\runs\fold3\exp\weights\best.pt
Ensemble 推論 using: c:\Users\mike\Desktop\aicup\runs\fold4\exp\weights\best.pt
Ensemble 推論 using: c:\Users\mike\Desktop\aicup\runs\fold5\exp\weights\best.pt
示例： [('patient0073_0050', [])]


In [15]:
# === 區塊 8（改版）：把兩資料夾的推論結果合併輸出成一個提交檔 ===
# 官規：每行
#   影像檔名(無 .png) 類別 分數 左上x 左上y 右下x 右下y
# 注意：
# - 沒有偵測的影像，不輸出任何行（不要寫空行）
# - 這裡直接把 preds_merged（兩資料夾合併後）寫成一個 .txt

def write_submission(preds: dict, out_path: Path):
    lines = []
    for stem, dets in preds.items(): 
        for (cls_id, score, x1, y1, x2, y2) in dets:
            # 類別固定 0（主動脈瓣）；分數保留 6 位小數，座標為整數
            lines.append(f"{stem} {cls_id} {score:.4f} {x1} {y1} {x2} {y2}")
    out_path.write_text("\n".join(lines), encoding="utf-8")
    return len(lines)

# 你可以自訂輸出檔名，例如：submission_test_two_dirs.txt
out_file = SUBMIT_DIR / "submission.txt"
n = write_submission(preds_merged, out_file)

print(f"✅ 已輸出提交檔：{out_file}")
print(f"   行數（偵測結果數量）＝ {n}")
print("   規則提醒：只輸出有偵測的影像，無偵測不寫任何東西。")


✅ 已輸出提交檔：c:\Users\mike\Desktop\aicup\submissions\submission.txt
   行數（偵測結果數量）＝ 3924
   規則提醒：只輸出有偵測的影像，無偵測不寫任何東西。
